In [1]:
import keras
import scipy
import numpy as np
import pandas as pd
import tensorflow as tf
from keras import layers

import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from numba import jit, float64, int64

plt.rc('font', size=12)          # controls default text sizes
plt.rc('axes', titlesize=12)     # fontsize of the axes title
plt.rc('axes', labelsize=15)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=12)    # fontsize of the tick labels
plt.rc('ytick', labelsize=12)    # fontsize of the tick labels
plt.rc('legend', fontsize=15)    # legend fontsize

In [2]:
@jit(nopython=True)
def Race(v, b, ndt=0, dt=0.001):
    x = np.array([0.0, 0.0])
    rt = 0
    ch = -1
    stop = False
    while not stop:
        x[0] += v[0]*dt + np.random.normal(0, 1)*np.sqrt(dt)
        x[1] += v[1]*dt + np.random.normal(0, 1)*np.sqrt(dt)
        rt += dt
        if x[0]>= b and x[1]<b:
            ch = 0.0
            stop = True
        elif x[1]>= b and x[0]<b:
            ch = 1.0
            stop = True
        elif x[0]>= b and x[1]>=b:
            rt = 0
            x = np.array([0.0, 0.0])
    return rt+ndt, ch

In [3]:
data_df = pd.read_csv('../../_00_data/Sepulveda2020_value_prep.csv', index_col=0)

min_rt = data_df.groupby(by='sbj')['rt'].min()

In [4]:
tr_df, ts_df = data_df.copy(), data_df.copy()

obs_train = np.asarray(data_df[['rt', 'choice', 'sbj']])
X_train = np.asarray(data_df[['item_value_0', 'item_value_1', 'gaze_0', 'gaze_1', 'goal']])

obs_test = np.asarray(data_df[['rt', 'choice', 'sbj']])
X_test = np.asarray(data_df[['item_value_0', 'item_value_1', 'gaze_0', 'gaze_1', 'goal']])

print(X_train.shape, X_test.shape)

(7289, 5) (7289, 5)


In [5]:
def fpt(t, b, v):
    return tf.cast(tf.math.greater(t, 0), tf.float32)*(b/(2*np.pi*t**3)**(1/2) * keras.backend.exp(-1/(2*t) * (v*t-b)**2)) + tf.cast(tf.math.less_equal(t, 0), tf.float32)*0.1**10

def phi(x):
    return 0.5*tf.math.erf(x/tf.sqrt(2.0)) + 0.5

def cdf(t, b, v):
    return tf.cast(tf.math.greater(t, 0), tf.float32)*(phi((v*t-b)/tf.sqrt(t)) + tf.exp(2*v*b)*phi((-v*t-b)/tf.sqrt(t))) + (tf.cast(tf.math.less_equal(t, 0), tf.float32)*0)

def G2(data, pred):

    sbj_indices = tf.cast(data[:, 2], tf.int32)

    # Gather values directly using TensorFlow operations
    gathered_min_rt = tf.gather(min_rt, sbj_indices)
    sigmoid_weights = tf.gather(model.layers[-1].weights[-1], sbj_indices)
    ndt = gathered_min_rt * tf.math.sigmoid(sigmoid_weights)

    thr_weights = tf.gather(model.layers[-1].weights[-2], sbj_indices)
    thr = tf.math.log(1 + tf.exp(thr_weights))

    scl_weights = tf.gather(model.layers[-1].weights[-3], sbj_indices)
    scl = tf.math.log(1 + tf.exp(scl_weights))

    ll1 = (1 - data[:, 1]) * fpt(data[:, 0]-ndt, thr, scl*pred[:, 0]) * (1.0 - cdf(data[:, 0]-ndt, thr, scl*pred[:, 1]))
    ll2 = (data[:, 1]) * fpt(data[:, 0]-ndt, thr, scl*pred[:, 1]) * (1.0 - cdf(data[:, 0]-ndt, thr, scl*pred[:, 0]))

    return -2*tf.reduce_sum(tf.math.log(tf.maximum(ll1 + ll2, 1e-14)), axis=-1)

In [6]:
class Out_layer(keras.layers.Layer):
    def __init__(self, units=2, input_dim=32, n_sbj_prms=10):
        super().__init__()
        self.w = self.add_weight(shape=(input_dim, units), initializer="random_normal", trainable=True)
        self.b = self.add_weight(shape=(units,), initializer="uniform", trainable=True)
        
        self.scaling = self.add_weight(shape=(n_sbj_prms,), initializer="uniform", trainable=True)
        self.threshold = self.add_weight(shape=(n_sbj_prms,), initializer="uniform", trainable=True)
        self.non_decision_time = self.add_weight(shape=(n_sbj_prms,), initializer="uniform", trainable=True)

    def call(self, inputs):
        return tf.math.log(1 + tf.exp(tf.matmul(inputs, self.w) + self.b))
    
def make_model(layers_neuron, input_shape, output_shape, activation, n_sbj_prms):
    keras.backend.clear_session()

    model = keras.Sequential()
    model.add(layers.Input(shape=(input_shape,)))
    
    for l in layers_neuron:
        model.add(layers.Dense(l, activation=activation))
    
    model.add(Out_layer(units=output_shape, input_dim=layers_neuron[-1], n_sbj_prms=n_sbj_prms))

    print(model.summary())
    
    return model

In [7]:
layers_neuron = [10, 10, 10, 8]
input_shape = 5
output_shape = 2
activation = 'tanh'
n_sbj = tr_df.sbj.nunique()

model = make_model(layers_neuron, input_shape, output_shape, activation, n_sbj)

model.build()
model.compile(optimizer="RMSprop", loss=G2)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                60        
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dense_2 (Dense)             (None, 10)                110       
                                                                 
 dense_3 (Dense)             (None, 8)                 88        
                                                                 
 out_layer (Out_layer)       (None, 2)                 111       
                                                                 
Total params: 479 (1.87 KB)
Trainable params: 479 (1.87 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [8]:
sbj_indices = tf.cast(obs_train[:, 2], tf.int32)
obs_train = tf.cast(obs_train, tf.float32)
X_train = tf.convert_to_tensor(X_train)
X_train = tf.cast(X_train, tf.float32)

min_rt = tf.convert_to_tensor(min_rt)
min_rt = tf.cast(min_rt, tf.float32)

In [9]:
history = model.fit(X_train, 
                    obs_train, 
                    shuffle=False, 
                    batch_size=X_train.shape[0], 
                    epochs=3500)

Epoch 1/3500
1/1 [==============================] - 1s 564ms/step - loss: 60212.7656
Epoch 2/3500
1/1 [==============================] - 0s 3ms/step - loss: 59919.1328
Epoch 3/3500
1/1 [==============================] - 0s 3ms/step - loss: 59713.7617
Epoch 4/3500
1/1 [==============================] - 0s 3ms/step - loss: 59542.1953
Epoch 5/3500
1/1 [==============================] - 0s 3ms/step - loss: 59388.4297
Epoch 6/3500
1/1 [==============================] - 0s 3ms/step - loss: 59245.3359
Epoch 7/3500
1/1 [==============================] - 0s 3ms/step - loss: 59109.1953
Epoch 8/3500
1/1 [==============================] - 0s 3ms/step - loss: 58977.8906
Epoch 9/3500
1/1 [==============================] - 0s 3ms/step - loss: 58850.0938
Epoch 10/3500
1/1 [==============================] - 0s 3ms/step - loss: 58724.9219
Epoch 11/3500
1/1 [==============================] - 0s 3ms/step - loss: 58601.7578
Epoch 12/3500
1/1 [==============================] - 0s 3ms/step - loss: 58480.1523

1/1 [==============================] - 0s 3ms/step - loss: 50561.4297
Epoch 99/3500
1/1 [==============================] - 0s 3ms/step - loss: 50503.0625
Epoch 100/3500
1/1 [==============================] - 0s 3ms/step - loss: 50445.3320
Epoch 101/3500
1/1 [==============================] - 0s 4ms/step - loss: 50388.2227
Epoch 102/3500
1/1 [==============================] - 0s 3ms/step - loss: 50331.7266
Epoch 103/3500
1/1 [==============================] - 0s 3ms/step - loss: 50275.8477
Epoch 104/3500
1/1 [==============================] - 0s 3ms/step - loss: 50220.5469
Epoch 105/3500
1/1 [==============================] - 0s 3ms/step - loss: 50165.8359
Epoch 106/3500
1/1 [==============================] - 0s 3ms/step - loss: 50111.6953
Epoch 107/3500
1/1 [==============================] - 0s 3ms/step - loss: 50058.1250
Epoch 108/3500
1/1 [==============================] - 0s 3ms/step - loss: 50005.1055
Epoch 109/3500
1/1 [==============================] - 0s 3ms/step - loss: 49952.6

1/1 [==============================] - 0s 3ms/step - loss: 46786.3672
Epoch 195/3500
1/1 [==============================] - 0s 4ms/step - loss: 46758.7656
Epoch 196/3500
1/1 [==============================] - 0s 3ms/step - loss: 46731.3047
Epoch 197/3500
1/1 [==============================] - 0s 4ms/step - loss: 46703.9844
Epoch 198/3500
1/1 [==============================] - 0s 3ms/step - loss: 46676.7969
Epoch 199/3500
1/1 [==============================] - 0s 4ms/step - loss: 46649.7422
Epoch 200/3500
1/1 [==============================] - 0s 3ms/step - loss: 46622.8203
Epoch 201/3500
1/1 [==============================] - 0s 4ms/step - loss: 46596.0312
Epoch 202/3500
1/1 [==============================] - 0s 3ms/step - loss: 46569.3711
Epoch 203/3500
1/1 [==============================] - 0s 3ms/step - loss: 46542.8281
Epoch 204/3500
1/1 [==============================] - 0s 3ms/step - loss: 46516.4258
Epoch 205/3500
1/1 [==============================] - 0s 4ms/step - loss: 46490.

1/1 [==============================] - 0s 4ms/step - loss: 44584.9688
Epoch 291/3500
1/1 [==============================] - 0s 4ms/step - loss: 44565.3594
Epoch 292/3500
1/1 [==============================] - 0s 4ms/step - loss: 44545.8047
Epoch 293/3500
1/1 [==============================] - 0s 3ms/step - loss: 44526.2930
Epoch 294/3500
1/1 [==============================] - 0s 4ms/step - loss: 44506.8203
Epoch 295/3500
1/1 [==============================] - 0s 3ms/step - loss: 44487.4023
Epoch 296/3500
1/1 [==============================] - 0s 4ms/step - loss: 44468.0312
Epoch 297/3500
1/1 [==============================] - 0s 4ms/step - loss: 44448.7031
Epoch 298/3500
1/1 [==============================] - 0s 4ms/step - loss: 44429.4180
Epoch 299/3500
1/1 [==============================] - 0s 4ms/step - loss: 44410.1758
Epoch 300/3500
1/1 [==============================] - 0s 4ms/step - loss: 44390.9805
Epoch 301/3500
1/1 [==============================] - 0s 4ms/step - loss: 44371.

1/1 [==============================] - 0s 4ms/step - loss: 42873.7891
Epoch 387/3500
1/1 [==============================] - 0s 4ms/step - loss: 42857.4219
Epoch 388/3500
1/1 [==============================] - 0s 5ms/step - loss: 42841.0781
Epoch 389/3500
1/1 [==============================] - 0s 4ms/step - loss: 42824.7500
Epoch 390/3500
1/1 [==============================] - 0s 4ms/step - loss: 42808.4609
Epoch 391/3500
1/1 [==============================] - 0s 4ms/step - loss: 42792.1875
Epoch 392/3500
1/1 [==============================] - 0s 4ms/step - loss: 42775.9414
Epoch 393/3500
1/1 [==============================] - 0s 4ms/step - loss: 42759.7188
Epoch 394/3500
1/1 [==============================] - 0s 4ms/step - loss: 42743.5195
Epoch 395/3500
1/1 [==============================] - 0s 4ms/step - loss: 42727.3438
Epoch 396/3500
1/1 [==============================] - 0s 3ms/step - loss: 42711.1914
Epoch 397/3500
1/1 [==============================] - 0s 3ms/step - loss: 42695.

...

1/1 [==============================] - 0s 4ms/step - loss: 26987.7012
Epoch 3053/3500
1/1 [==============================] - 0s 4ms/step - loss: 26986.9023
Epoch 3054/3500
1/1 [==============================] - 0s 4ms/step - loss: 26986.0742
Epoch 3055/3500
1/1 [==============================] - 0s 4ms/step - loss: 26985.3008
Epoch 3056/3500
1/1 [==============================] - 0s 4ms/step - loss: 26984.7891
Epoch 3057/3500
1/1 [==============================] - 0s 4ms/step - loss: 26985.0859
Epoch 3058/3500
1/1 [==============================] - 0s 4ms/step - loss: 26985.5605
Epoch 3059/3500
1/1 [==============================] - 0s 3ms/step - loss: 26985.3086
Epoch 3060/3500
1/1 [==============================] - 0s 4ms/step - loss: 26985.5000
Epoch 3061/3500
1/1 [==============================] - 0s 3ms/step - loss: 26984.5332
Epoch 3062/3500
1/1 [==============================] - 0s 4ms/step - loss: 26983.2773
Epoch 3063/3500
1/1 [==============================] - 0s 4ms/step - l

1/1 [==============================] - 0s 3ms/step - loss: 26961.9590
Epoch 3148/3500
1/1 [==============================] - 0s 3ms/step - loss: 26960.7773
Epoch 3149/3500
1/1 [==============================] - 0s 3ms/step - loss: 26961.0586
Epoch 3150/3500
1/1 [==============================] - 0s 4ms/step - loss: 26958.8379
Epoch 3151/3500
1/1 [==============================] - 0s 3ms/step - loss: 26959.3906
Epoch 3152/3500
1/1 [==============================] - 0s 3ms/step - loss: 26958.2852
Epoch 3153/3500
1/1 [==============================] - 0s 4ms/step - loss: 26958.5625
Epoch 3154/3500
1/1 [==============================] - 0s 3ms/step - loss: 26959.4375
Epoch 3155/3500
1/1 [==============================] - 0s 4ms/step - loss: 26960.0234
Epoch 3156/3500
1/1 [==============================] - 0s 4ms/step - loss: 26958.6914
Epoch 3157/3500
1/1 [==============================] - 0s 3ms/step - loss: 26958.7344
Epoch 3158/3500
1/1 [==============================] - 0s 4ms/step - l

1/1 [==============================] - 0s 4ms/step - loss: 26939.9707
Epoch 3243/3500
1/1 [==============================] - 0s 4ms/step - loss: 26939.2207
Epoch 3244/3500
1/1 [==============================] - 0s 4ms/step - loss: 26940.2188
Epoch 3245/3500
1/1 [==============================] - 0s 4ms/step - loss: 26939.7227
Epoch 3246/3500
1/1 [==============================] - 0s 4ms/step - loss: 26937.7910
Epoch 3247/3500
1/1 [==============================] - 0s 3ms/step - loss: 26937.8750
Epoch 3248/3500
1/1 [==============================] - 0s 4ms/step - loss: 26935.4570
Epoch 3249/3500
1/1 [==============================] - 0s 4ms/step - loss: 26935.9844
Epoch 3250/3500
1/1 [==============================] - 0s 4ms/step - loss: 26935.4297
Epoch 3251/3500
1/1 [==============================] - 0s 4ms/step - loss: 26936.2188
Epoch 3252/3500
1/1 [==============================] - 0s 4ms/step - loss: 26937.1289
Epoch 3253/3500
1/1 [==============================] - 0s 4ms/step - l

1/1 [==============================] - 0s 5ms/step - loss: 26923.7637
Epoch 3338/3500
1/1 [==============================] - 0s 4ms/step - loss: 26922.9023
Epoch 3339/3500
1/1 [==============================] - 0s 4ms/step - loss: 26922.5391
Epoch 3340/3500
1/1 [==============================] - 0s 5ms/step - loss: 26919.9883
Epoch 3341/3500
1/1 [==============================] - 0s 4ms/step - loss: 26918.7266
Epoch 3342/3500
1/1 [==============================] - 0s 5ms/step - loss: 26920.2344
Epoch 3343/3500
1/1 [==============================] - 0s 4ms/step - loss: 26920.6094
Epoch 3344/3500
1/1 [==============================] - 0s 5ms/step - loss: 26919.9551
Epoch 3345/3500
1/1 [==============================] - 0s 5ms/step - loss: 26919.4727
Epoch 3346/3500
1/1 [==============================] - 0s 5ms/step - loss: 26919.8711
Epoch 3347/3500
1/1 [==============================] - 0s 5ms/step - loss: 26921.1562
Epoch 3348/3500
1/1 [==============================] - 0s 5ms/step - l

1/1 [==============================] - 0s 4ms/step - loss: 26907.1484
Epoch 3433/3500
1/1 [==============================] - 0s 4ms/step - loss: 26907.0312
Epoch 3434/3500
1/1 [==============================] - 0s 4ms/step - loss: 26906.8086
Epoch 3435/3500
1/1 [==============================] - 0s 4ms/step - loss: 26905.7637
Epoch 3436/3500
1/1 [==============================] - 0s 4ms/step - loss: 26903.7070
Epoch 3437/3500
1/1 [==============================] - 0s 4ms/step - loss: 26903.9453
Epoch 3438/3500
1/1 [==============================] - 0s 4ms/step - loss: 26905.4160
Epoch 3439/3500
1/1 [==============================] - 0s 4ms/step - loss: 26904.8789
Epoch 3440/3500
1/1 [==============================] - 0s 5ms/step - loss: 26905.9434
Epoch 3441/3500
1/1 [==============================] - 0s 4ms/step - loss: 26905.5039
Epoch 3442/3500
1/1 [==============================] - 0s 4ms/step - loss: 26905.3242
Epoch 3443/3500
1/1 [==============================] - 0s 4ms/step - l

In [10]:
pred_v = model.predict(X_test)
threshold = np.asanyarray(tf.math.log(1 + tf.exp(model.layers[-1].weights[-2])))
ndt = np.asanyarray(np.asanyarray(min_rt)*tf.math.sigmoid(model.layers[-1].weights[-1]))

np.save('input_value.npy', X_test)
np.save('output_value.npy', pred_v)

228/228 [==============================] - 0s 369us/step


In [11]:
pred_bhv = {'sbj': [],
            'rt': [],
            'ch': [],
            'goal':[],
            'value_diff': [],
            'gaze_diff': []}

scaling = np.asanyarray(tf.math.log(1 + tf.exp(model.layers[-1].weights[-3])))
scaling = np.asanyarray([scaling[i] for i in ts_df['sbj']])

for n in range(20):
    for s in range(ts_df.shape[0]):
        rt, ch = Race(scaling[s]*pred_v[s,:], threshold[ts_df.sbj[s]], ndt[ts_df.sbj[s]])
        pred_bhv['sbj'].append(ts_df.sbj[s])
        pred_bhv['rt'].append(rt)
        pred_bhv['ch'].append(ch)
        pred_bhv['goal'].append(ts_df.goal[s])
        pred_bhv['value_diff'].append(ts_df['Right Value - Left Value'][s])
        pred_bhv['gaze_diff'].append(ts_df['Dwell Time Advantage (R - L)'][s])
        
pred_bhv = pd.DataFrame(pred_bhv)
pred_bhv.to_csv('Sp2020_value_complete.csv')